In [ ]:
import random
from functools import lru_cache
from statistics import mean, median
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from enum import IntEnum


class Individual():
    '''個体
    
    ・遺伝情報
      ・長さ95のビット列で表現する（0 ~ 2^95 - 1）
      ・それぞれ以下に発現する
        ・first_char(5bit)：A~Zを含む1文字
        ・first_str_num(3bit)：0~7までの数字
        ・first_str(5bit * 8)：a~zを含む8文字
        ・first_delim(2bit)：「,」「.」「!」「?」＋スペース
        ・second_str_num(3bit)：0~7までの数字
        ・second_str(5bit * 8)：a~zを含む8文字
        ・last_delim(2bit)：「.」「!」「?」「!?」
    '''
    _mask_delim = 0b11
    _shift_delim = 2
    
    _mask_num = 0b111
    _shift_num = 3
    
    _mask_char = 0b11111
    _shift_char = 5
    
    _gene_size = 95
    
    def __init__(self, gene=None):
        if gene is None:
            self.gene = random.getrandbits(self._gene_size)
        else:
            self.gene = gene
        
        self._parse_gene()
        self._express()
        self._calc_fitness()
    
    def _parse_gene(self):
        '''遺伝情報を読み解く'''
        gene = self.gene
        
        # 最後のデリミタ
        self._second_delim = gene & self._mask_delim
        gene = gene >> self._shift_delim
        
        # 後半の文字列
        self._second_str = []
        for _ in range(8):
            self._second_str.append(gene & self._mask_char)
            gene = gene >> self._shift_char
        
        # 後半の文字数
        self._second_str_num = gene & self._mask_num
        gene = gene >> self._shift_num
        
        # 最初のデリミタ
        self._first_delim = gene & self._mask_delim
        gene = gene >> self._shift_delim
        
        # 前半の文字列
        self._first_str = []
        for _ in range(8):
            self._first_str.append(gene & self._mask_char)
            gene = gene >> self._shift_char
        
        # 前半の文字数
        self._first_str_num = gene & self._mask_num
        gene = gene >> self._shift_num
        
        # 1文字目
        self._first_char = gene & self._mask_char
    
    def _express(self):
        '''遺伝子から個体へと発現する'''
        self.body = []
        
        # 1文字目
        self.body.append(chr(self._first_char + 59))
        
        # 前半の文字列
        for i in range(self._first_str_num + 1):
            self.body.append(chr(self._first_str[i] + 91))
        
        # 最初のデリミタ
        if self._first_delim == 0:
            self.body.append(', ')
        elif self._first_delim == 1:
            self.body.append('. ')
        elif self._first_delim == 2:
            self.body.append('! ')
        elif self._first_delim == 3:
            self.body.append('? ')
        
        # 後半の文字列
        for i in range(self._second_str_num + 1):
            self.body.append(chr(self._second_str[i] + 91))
        
        # 最後のデリミタ
        if self._second_delim == 0:
            self.body.append('.')
        elif self._second_delim == 1:
            self.body.append('!')
        elif self._second_delim == 2:
            self.body.append('?')
        elif self._second_delim == 3:
            self.body.append('!?')
        
        # 合体
        self.body = ''.join(self.body)
    
    def _calc_fitness(self):
        '''適応度を計算する'''
        self.fitness = lss(self.body, 'Hello, world!')
    
    def _is_valid_operand(self, other):
        return hasattr(other, 'fitness')
    
    def __eq__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return self.fitness == other.fitness
    
    def __ne__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return self.fitness != other.fitness
    
    def __lt__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return self.fitness < other.fitness
    
    def __le__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return self.fitness <= other.fitness
    
    def __gt__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return self.fitness > other.fitness
    
    def __ge__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return self.fitness >= other.fitness
    
    def __hash__(self):
        return hash(self.fitness)
    
    def __repr__(self):
        return f'{self.body}({self.fitness})'
    
    def __str__(self):
        return self.body
    
    def __format__(self, format_spec):
        return self.body
    
    def __bool__(self):
        return True if self.fitness == 1.0 else False
    
    def mate(self, other):
        '''子供を作る'''
        if not hasattr(other, 'gene'):
            return NotImplemented
        
        child_gene = 0
        self_gene = self.gene
        other_gene = other.gene
        
        # まずは、子供の遺伝情報の断片を集める
        # 最後のデリミタ
        self_second_delim = self_gene & self._mask_delim
        other_second_delim = other_gene & self._mask_delim
        child_second_delim = random.choice([self_second_delim, other_second_delim])
        self_gene = self_gene >> self._shift_delim
        other_gene = other_gene >> self._shift_delim
        
        # 後半の文字列
        child_second_str = []
        for _ in range(8):
            self_second_str_char = self_gene & self._mask_char
            other_second_str_char = other_gene & self._mask_char
            child_second_str_char = random.choice([self_second_str_char, other_second_str_char])
            child_second_str.append(child_second_str_char)
            self_gene = self_gene >> self._shift_char
            other_gene = other_gene >> self._shift_char
        child_second_str.reverse()
        
        # 後半の文字数
        self_second_str_num = self_gene & self._mask_num
        other_second_str_num = other_gene & self._mask_num
        child_second_str_num = random.choice([self_second_str_num, other_second_str_num])
        self_gene = self_gene >> self._shift_num
        other_gene = other_gene >> self._shift_num
        
        # 最初のデリミタ
        self_first_delim = self_gene & self._mask_delim
        other_first_delim = other_gene & self._mask_delim
        child_first_delim = random.choice([self_first_delim, other_first_delim])
        self_gene = self_gene >> self._shift_delim
        other_gene = other_gene >> self._shift_delim
        
        # 前半の文字列
        child_first_str = []
        for _ in range(8):
            self_first_str_char = self_gene & self._mask_char
            other_first_str_char = other_gene & self._mask_char
            child_first_str_char = random.choice([self_first_str_char, other_first_str_char])
            child_first_str.append(child_first_str_char)
            self_gene = self_gene >> self._shift_char
            other_gene = other_gene >> self._shift_char
        child_first_str.reverse()
        
        # 前半の文字数
        self_first_str_num = self_gene & self._mask_num
        other_first_str_num = other_gene & self._mask_num
        child_first_str_num = random.choice([self_first_str_num, other_first_str_num])
        self_gene = self_gene >> self._shift_num
        other_gene = other_gene >> self._shift_num
        
        # 1文字目
        self_first_char = self_gene & self._mask_char
        other_first_char = other_gene & self._mask_char
        child_first_char = random.choice([self_first_char, other_first_char])
        
        # 集めた子供の遺伝情報の断片を一つにする
        # 1文字目
        child_gene = child_gene | child_first_char
        
        # 前半の文字数
        child_gene = child_gene << self._shift_num
        child_gene = child_gene | child_first_str_num
        
        # 前半の文字列
        for child_first_str_char in child_first_str:
            child_gene = child_gene << self._shift_char
            child_gene = child_gene | child_first_str_char
        
        # 最初のデリミタ
        child_gene = child_gene << self._shift_delim
        child_gene = child_gene | child_first_delim
        
        # 後半の文字数
        child_gene = child_gene << self._shift_num
        child_gene = child_gene | child_second_str_num
        
        # 後半の文字列
        for child_second_str_char in child_second_str:
            child_gene = child_gene << self._shift_char
            child_gene = child_gene | child_second_str_char
        
        # 最後のデリミタ
        child_gene = child_gene << self._shift_delim
        child_gene = child_gene | child_second_delim
        
        return Individual(child_gene)
    
    def mutate(self, p):
        '''突然変異する'''
        if random.random() <= p:
            mask = random.getrandbits(self._gene_size)
            self.gene = self.gene ^ mask
            self._parse_gene()
            self._express()
            self._calc_fitness()
        
        return self


class Population():
    '''個体の集団'''
    def __init__(self, pool_size):
        self.pool_size = pool_size
        self.generation = [Individual() for _ in range(self.pool_size)]
        self.generation.sort(reverse=True)
        self.generation_number = 0
    
    def next_generation(self):
        '''次世代を生み出す'''
        self.generation_number += 1
        
        # エリートと非エリートに分ける
        pareto = self.pool_size // 5
        self.elites = self.generation[: pareto]
        self.non_elites = self.generation[pareto :]
        
        # エリートのみが繁殖に成功する（ハーレム）
        self.children = []
        for parent1 in self.elites:
            while True:
                parent2 = random.choice(self.generation)
                if parent1 is parent2:
                    continue
                else:
                    break
            
            for _ in range(2):
                self.children.append(parent1.mate(parent2))
        
        # 非エリートは80%の確率で突然変異する
        self.non_elites = [i.mutate(0.8) for i in self.non_elites]
        
        # エリートは20%の確率で突然変異する
        self.elites = [i.mutate(0.2) for i in self.elites]
        
        # 次世代を絞り込む
        next_gen = self.elites + self.children + self.non_elites
        next_gen.sort(reverse=True)
        self.generation = next_gen[: self.pool_size]
        
        # 結果
        min_fitness = self.generation[-1].fitness
        max_fitness = self.generation[0].fitness
        mean_fitness = mean(i.fitness for i in self.generation)
        median_fitness = median(i.fitness for i in self.generation)
        
        return self.generation[0].body, min_fitness, max_fitness, mean_fitness, median_fitness


@lru_cache(maxsize=None)
def ld(s, t):
    '''編集距離（レーベンシュタイン距離）を計算する'''
    if not s: return len(t)
    if not t: return len(s)
    if s[0] == t[0]: return ld(s[1:], t[1:])
    l1 = ld(s, t[1:])
    l2 = ld(s[1:], t)
    l3 = ld(s[1:], t[1:])
    return 1 + min(l1, l2, l3)


def lss(s, t):
    '''類似度を計算する（編集距離を標準化し線形変換する）'''
    return -(ld(s, t) / max(len(s), len(t))) + 1


class Fitness(IntEnum):
    '''適応度についてのEnum'''
    min = 0
    max = 1
    mean = 2
    median = 3


def hello_world_with_ga():
    '''遺伝的アルゴリズムでHello world!'''
    p = Population(1000)
    for i in range(100):
        body, *fitness = p.next_generation()
        if fitness[Fitness.max] == 1:
            break
    print(body)


def watch_hello_world():
    '''進化の過程を見守る'''
    h = [[], [], [], []]
    p = Population(1000)
    
    for _ in range(100):
        print(f'{p.generation_number} : {p.generation[0]}')
        body, *fitness = p.next_generation()
        for f in Fitness:
            h[f].append(fitness[f])
        if fitness[Fitness.max] == 1:
            print(f'{p.generation_number} : {p.generation[0]}')
            break
    
    x = range(1, len(h[Fitness.min]) + 1)
    plt.figure()
    plt.plot(x, h[Fitness.min], marker='.', label='min_fitness')
    plt.plot(x, h[Fitness.max], marker='.', label='max_fitness')
    plt.plot(x, h[Fitness.mean], marker='.', label='mean_fitness')
    plt.plot(x, h[Fitness.median], marker='.', label='median_fitness')
    plt.legend(loc='best', fontsize=10)
    plt.grid()
    plt.xlabel('generation')
    plt.ylabel('fitness')
    plt.title(body)
    plt.gca().get_xaxis().set_major_locator(ticker.MaxNLocator(integer=True))
    plt.show()


hello_world_with_ga()
watch_hello_world()